In [170]:
import geopandas as gpd
import gdal as gd
import matplotlib.pyplot as plt
from matplotlib.pyplot import axis
import pandas as pd

## Inputs and Adjustments

In [171]:
# csv of the snow depth measurements 
depths = 'SnowDepthMeasurements_TVDW.csv'

## Import Data

In [189]:
# Read the files that Thomas Sent
df0 = gpd.read_file(depths)

# collected files from 20210211
df1 = pd.read_csv('LOG01.TXT')
df1.to_csv(r'2021_02_11_probe.csv', index = None)
df1 = gpd.read_file('2021_02_11_probe.csv')


# collect second log
df2 = pd.read_csv('LOG02.TXT')
df2.to_csv(r'2021_02_11_probe2.csv', index = None)
df2 = gpd.read_file('2021_02_11_probe2.csv')

df0

,Date,Time,Latitude,Longitude,Elevation,Depth,geometry
0,1/8/2021,1:41:00 PM,43.7396278,-116.1203842,5254,0.81,None
1,1/8/2021,1:42:00 PM,43.7396202,-116.1203613,5254,0.56,None
2,1/8/2021,1:43:00 PM,43.7396164,-116.1203461,5254,0.58,None
3,1/8/2021,1:44:00 PM,43.7396049,-116.1202927,5253,0.56,None
4,1/8/2021,1:45:00 PM,43.7396011,-116.1202698,5252,0.84,None
...,...,...,...,...,...,...,...
148,1/20/2021,2:56:15 PM,43.7356644,-116.1228409,5150,0.75,None
149,1/20/2021,2:57:03 PM,43.7357330,-116.1228714,5146,0.87,None
150,1/20/2021,2:58:50 PM,43.7356300,-116.1229095,5144,0.87,None
151,1/20/2021,2:59:47 PM,43.7354584,-116.1229935,5140,0.55,None


In [188]:

df_old = df1

# turn appropriate columsn to intergers
df_old = df_old.astype({'DistanceAvg' : 'int' , 
    'Distance1' : 'int' ,
    'Distance2' : 'int' ,
    'Distance3' : 'int' ,
    'Distance4' : 'int' ,
    'Distance5' : 'int' ,
    'altitude' : 'int' }) 
df_old = df_old[df_old['DistanceAvg'] != 0] # take out any points where the average is 0

# set the titles of the new dataframe
df_new = pd.DataFrame( columns = ['Date' , 'Time' , 'Latitude' , 'Longitude' , 'Elevation' , 'Depth' , 'geometry'])

# pull out lat and long
df_new['Latitude'] = df_old['latitude']
df_new['Longitude'] = df_old['longitude']
df_new['Elevation'] = df_old['altitude']
df_new['geometry'] = df_old['geometry']


# extract the date from the original dataframe 
df_old['startGPSTime'] = pd.to_datetime(df_old['startGPSTime']) # turn the date to a datetime format
df_new['Date'] = df_old['startGPSTime'].dt.strftime('%m/%d/%Y') # properly format date to match previous dataframe
df_new['Time'] = df_old['startGPSTime'].dt.strftime('%H:%M:%S') # properly format time to match, taking 

########################## Really think something is up here and I need to talk to thomas about this because most of the heights are greater than the probe#######################################################
# calculate the Snow Heights
dst =  df_old[['Distance1' , 'Distance2' , 'Distance3' , 'Distance4' , 'Distance5']].copy().astype('int') # create a frame of heights
probe_height = dst.iloc[0].mode().astype('int') # pull probe height from mode of measurement 0
probe_height = probe_height[0]/100 # pull out the interger from the series and convert to m

raw = dst.median(axis = 'columns').astype('int') # pull out the median from the height measurements
raw = raw/100 # convert to m
# df_new['Depth'] -= probe_height # subtract probe height from measured distance to get the height of snow

# data cleaning
df_new['Depth'] = probe_height-raw # difference between probe height and raw measurements
df_new['Depth'] = raw - probe_height # difference between raw measurements and  probe height 
df_new = df_new[df_new['Depth'] >= 0] # take out any points where the average is 0
df_new = df_new[df_new['Depth'] <= probe_height] # take out any points where the measurement is larger than the probe height

df_new

,Date,Time,Latitude,Longitude,Elevation,Depth,geometry
0,02/11/2021,15:04:47,43.763237,-116.1047058,5335,0.00,None
2,02/11/2021,15:05:04,43.7632332,-116.1047058,5336,1.13,None
